In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import bioinfokit as bio
import seaborn as sns

ModuleNotFoundError: No module named 'bioinfokit'

In [ ]:
dataset = pd.read_csv("mcdonalds.csv")
dataset

In [ ]:
dataset.info(3)

In [ ]:
dataset.shape

In [ ]:
dataset.mean()

In [ ]:
Gender = ["Female", "Male"]
Color = ["pink", "cyan"]
Size = dataset["Gender"].value_counts()
plt.pie(Size, labels=Gender, colors=Color, autopct="%.2f%%")
plt.legend()
plt.show()

In [ ]:
f = sns.countplot(x=dataset["Age"], palette='hsv')
f.bar_label(f.containers[0])
plt.rcParams['figure.figsize'] = (25, 8)

In [ ]:
dataset["Like"]=dataset["Like"].replace({'I hate it!-5': '-5', 'I love it!+5':'+5'})
dataset.dtypes

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

cols=["yummy", "convenient", "spicy", "fattening", "greasy", "fast", "cheap", "tasty", "expensive", "healthy", "disgusting"]
    
for i in cols:
    dataset[i]=le.fit_transform(dataset[i])


In [ ]:
dataset

In [ ]:
dataset.hist()

In [ ]:
x = dataset.loc[:, cols]
x

In [ ]:
from sklearn.decomposition import PCA
from sklearn import preprocessing

pca_data = preprocessing.scale(x)

pca = PCA(n_components=11)
pc = pca.fit_transform(x)
names=["pc1", "pc2", "pc3", "pc4", "pc5", "pc6", "pc7", "pc8", "pc9", "pc10", "pc11"]
pf = pd.DataFrame(data=pc, columns=names)
pf

In [ ]:
std_dev = []
for i in names:
    std_dev.append(np.std(pf[i]))
    
print("Standard Deviation")
np.array(std_dev)

In [ ]:
pov = pca.explained_variance_ratio_
print("Proportion of Variance")
pov

In [ ]:
cp = np.cumsum(pca.explained_variance_ratio_)
print("Cumulative Proportion")
cp

In [ ]:
components = pca.components_
num_pc = pca.n_features_
pc_name = ["PC"+str(i) for i in range(1, num_pc+1)]
loadings_df = pd.DataFrame

In [ ]:
from sklearn.cluster import KMeans

wcss=[]
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(pf)
    wcss.append(kmeans.inertia_)

plt.plot(range(1, 11), wcss)
plt.title("Elbow method")
plt.xlabel("Number of clusters")
plt.ylabel("WCSS")
plt.show()

In [ ]:
#K-means clustering 

kmeans = KMeans(n_clusters=4, init='k-means++', random_state=0).fit(x)
dataset['cluster_num'] = kmeans.labels_ #adding to df
print (kmeans.labels_) #Label assigned for each data point
print (kmeans.inertia_) #gives within-cluster sum of squares. 
print(kmeans.n_iter_) #number of iterations that k-means algorithm runs to get a minimum within-cluster sum of squares
print(kmeans.cluster_centers_) #Location of the centroids on each cluster. 

In [ ]:
from collections import Counter
Counter(kmeans.labels_)

In [ ]:
# correlation coefficient between original variables and the component

loadings = pca.components_
num_pc = pca.n_features_
pc_list = ["PC"+str(i) for i in list(range(1, num_pc+1))]
loadings_df = pd.DataFrame.from_dict(dict(zip(pc_list, loadings)))
loadings_df['variable'] = x.columns.values
loadings_df = loadings_df.set_index('variable')
loadings_df

In [ ]:
#Scree plot (Elbow test)- PCA
from bioinfokit.visuz import cluster
cluster.screeplot(obj=[pc_list, pca.explained_variance_ratio_],show=True,dim=(10,5))# get PC scores
pca_scores = PCA().fit_transform(x)

# get 2D biplot
cluster.biplot(cscore=pca_scores, loadings=loadings, labels=dataset.columns.values, var1=round(pca.explained_variance_ratio_[0]*100, 2),
    var2=round(pca.explained_variance_ratio_[1]*100, 2),show=True,dim=(10,5))

In [ ]:
f = sns.countplot(x=dataset["cluster_num"], palette='hsv')
f.bar_label(f.containers[0])
plt.rcParams['figure.figsize'] = (25, 8)
# print(len(dataset['cluster_num'][1]))

In [ ]:
dataset1=pd.get_dummies(dataset, prefix=['cluster_num'], columns=['cluster_num'])


In [ ]:
dataset1

In [ ]:
#Gender
# df['Gender'] = LabelEncoder().fit_transform(df['Gender'])
result=[]
for i in cols:
    Gender = dataset.groupby(['cluster_num'])[i].mean()
    Gender = Gender.to_frame().reset_index()
    result.append(Gender)

In [ ]:
for i in range(len(result)):
    print(pd.DataFrame(result[i]))

In [ ]:
dataset1=pd.get_dummies(dataset, prefix=['VisitFrequency'], columns=['VisitFrequency'])


In [ ]:
dataset1

In [ ]:
for i in dataset1.columns:
    sns.countplot(y=dataset1[i], hue=dataset1['cluster_num'], data=dataset1)
#     plt.rcParams['figure.figsize'] = (25, 8)
    plt.rcParams['figure.dpi'] = 140
    plt.rcParams['font.size'] = 20
    plt.show()

# for i in ['cluster_num_0', 'cluster_num_1', 'cluster_num_2', 'cluster_num_3']:
#     for j in cols:
#         sns.countplot(x=dataset1[i], hue=dataset1[j])
#         plt.show()

In [ ]:
sns.scatterplot(data=pf, x="pc1", y="pc2", hue=kmeans.labels_)
plt.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], 
            marker="X", c="r", s=80, label="centroids")
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['font.size'] = 16
plt.legend()
plt.show()

In [ ]:
from statsmodels.graphics.mosaicplot import mosaic
from itertools import product
crosstab =pd.crosstab(dataset['cluster_num'],dataset['Like'])
#Reordering cols
crosstab = crosstab[['-5','-4','-3','-2','-1','0','+1','+2','+3','+4','+5']]
crosstab 

In [ ]:
mosaic(crosstab.stack())
plt.rcParams['figure.figsize'] = (5, 5)
plt.rcParams['font.size'] = 5
plt.show()

In [ ]:
crosstab =pd.crosstab(dataset['cluster_num'],dataset['Gender'])
#Reordering cols
crosstab = crosstab[['Female', 'Male']]
mosaic(crosstab.stack())
plt.show()

In [ ]:
sns.boxplot(x='cluster_num', y='Age', data=dataset1)
